In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Platform (Unified) SDK: Training a custom image classification model using a training pipeline with a managed dataset input

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_image_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/notebooks/deepdive/custom/ucaip_customjob_image_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Overview


This tutorial demonstrates how to use the AI Platform (Unified) Python SDK to train a custom image classification model using a [training pipeline job](https://cloud.google.com/ai-platform-unified/docs/training/create-training-pipeline) with a managed dataset input.

### Dataset

The dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of flower an image is from a class of five flowers: daisy, dandelion, rose, sunflower, tulip.

### Objectives


- Creating a AI Platform (Unified) dataset.
- Creating a training pipeline with a custom job training step and an AI Platform dataset input.
- Starting the training pipeline job.
- Monitoring the training pipeline job.
- Deploy the model to a serving endpoint.
- Make a prediction(s).
- Undeploy the model.

### Costs 

This tutorial uses billable components of Google Cloud Platform (GCP):

* Cloud AI Platform
* Cloud Storage

Learn about [Cloud AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Environment

This notebook was developed and tested using [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks) using the TensorFlow 2.3 image.

## Before you begin

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform APIs, Compute Engine APIs and Container Registry API.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,containerregistry.googleapis.com)

4. Follow the instructions in the repos' [README file](https://github.com/jarokaz/ucaip/blob/main/README.md) to provision an instance of AI Platform Notebooks and install Cloud AI Platform (Unified) SDK.



#### Set your Project ID

**If you don't know your project ID**, you might be able to get your project ID using `gcloud` command by executing the second cell below.

In [1]:
PROJECT_ID = "jk-mlops-dev" 

In [2]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You can
not use a Multi-Regional Storage bucket for training with AI Platform.

In [3]:
REGION = 'us-central1' 

### Create a Cloud Storage bucket

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

In [4]:
BUCKET_NAME = "jk-ucaip-demos" 

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [5]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://jk-ucaip-demos/...
ServiceException: 409 Bucket jk-ucaip-demos already exists.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [8]:
! gsutil ls -al gs://$BUCKET_NAME

### Import libraries and define constants

In [9]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Import AI Platform (Unified) SDK

Import the AI Platform (Unified) SDK into our python environment.

In [10]:
import json
import os
import sys
import time

import tensorflow as tf
import tensorflow_io as tfio

from google.cloud.aiplatform import gapic as aip

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

from datetime import datetime

### AI Platform (Unified) constants

Let's now setup some constants for AI Platform (Unified):

#### Endpoints

- `API_ENDPOINT`: The AI Platform (Unified) API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The AI Platform (Unified) API service endpoint for prediction.
- `PARENT`: The AI Platform (Unified) location root path for dataset, model and endpoint resources.

In [11]:
# API Endpoint
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"
API_PREDICT_ENDPOINT = "us-central1-prediction-aiplatform.googleapis.com"

# AI Platform (Unified) location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# Default timeout for API calls
TIMEOUT = 60

#### Dataset schemas

AI Platform supports four dataset types: tabular, text, image and video. The same dataset type can be used for multiple ML tasks. For example, the image dataset type can be used for single-label classification, multi-label classification or object detection. In this sample, you will create an image dataset for the single-label classificatin task.

The dataset type and the ML task are specified by a set of pre-defined YAML based schemas provided by AI Platform.

In [12]:
IMAGE_SCHEMA = 'gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml' 
IMPORT_SCHEMA_IMAGE_CLASSIFICATION = 'gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml'

#### Training pipeline schemas

When configuring a training pipeline you need to specify a type of a training task - a custom job or AutoML training. In addition, if your pipeline uses an AI Platform dataset as input, you have to specify a type of annotations to use for training - an AI Platform dataset can have multiple annotation types attached to the dataset. 

In our case, we will configure a pipeline with a custom training task and image dataset with image classification annotations.

In [13]:
ANNOTATION_SCHEMA = 'gs://google-cloud-aiplatform/schema/dataset/annotation/image_classification_1.0.0.yaml'
CUSTOM_TRAINING_TASK_DEFINITION = 'gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml'

#### Training and deployment container images

These constants define a base image for a custom training and a serving image.

In [14]:
TRAIN_BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-3:latest'
DEPLOY_IMAGE = 'gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

#### Machine configurations for training and deployment

In [15]:
TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

TRAIN_MACHINE_TYPE = 'n1-standard-4'
DEPLOY_MACHINE_TYPE = 'n1-standard-4'

# Tutorial

## Clients

The AI Platform (Unified) SDK works as a client/server model. On your side, the python script, you will create a client that sends requests and receives responses from the server -- AI Platform.

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Prediction Service for serving. *Note*, prediction has a different service endpoint.


In [17]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}
predict_client_options = {"api_endpoint": API_PREDICT_ENDPOINT}

dataset_client = aip.DatasetServiceClient(client_options=client_options)
model_client = aip.ModelServiceClient(client_options=client_options)
pipeline_client = aip.PipelineServiceClient(client_options=client_options)
endpoint_client = aip.EndpointServiceClient(client_options=client_options)
prediction_client = aip.PredictionServiceClient(client_options=predict_client_options)

## Creating an AI Platform dataset

Creating an AI Platform dataset is a two-step process. The first step is to create an empty dataset. During the first step you define the dataset type. The second step is to import the data to the dataset. This is when you specify the ML task supported by the imported data.


### Create an empty image dataset

Both creating a dataset and importing data are long running operations in AI Platform. The long running operations use asynchronous calls. An asynchronous call does not block a caller and returns an `operation` object that can by subsequently used to monitor/control the operation by invoking methods exposed by the object:


| Method      | Description |
| ----------- | ----------- |
| result()    | Waits for the operation to complete and returns a result object in JSON format.      |
| running()   | Returns True/False on whether the operation is still running.        |
| done()      | Returns True/False on whether the operation is completed. |
| canceled()  | Returns True/False on whether the operation was canceled. |
| cancel()    | Cancels the operation (this may take up to 30 seconds). |



In [18]:
display_name = 'flowers-dataset'

dataset = aip.Dataset(
    display_name=display_name,
    metadata_schema_uri=IMAGE_SCHEMA,
    labels=None
)

operation = dataset_client.create_dataset(parent=PARENT, dataset=dataset)
print("Long running operation:", operation.operation.name)
response = operation.result(timeout=TIMEOUT)
print(response)
dataset_name = response.name

Long running operation: projects/895222332033/locations/us-central1/datasets/7399009567490703360/operations/2047015773011968000
name: "projects/895222332033/locations/us-central1/datasets/7399009567490703360"
display_name: "flowers-dataset"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
labels {
  key: "aiplatform.googleapis.com/dataset_metadata_schema"
  value: "IMAGE"
}
metadata {
  struct_value {
    fields {
      key: "dataItemSchemaUri"
      value {
        string_value: "gs://google-cloud-aiplatform/schema/dataset/dataitem/image_1.0.0.yaml"
      }
    }
  }
}



### Prepare data for import

The data to be imported to an AI Platform image dataset must meet the following requirements:

- Images must be stored in a Cloud Storage bucket.
- Each image file must be in an image format (PNG, JPEG, BMP, ...).
- There must be an index file stored in your Cloud Storage bucket that contains the path and annotations for each image.
- The index file must be either CSV or JSONL.

#### CSV

For image classification, the CSV index file must have the following format:

- No heading
- First column is the Cloud Storage path to the image.
- Second column is the label.

#### JSONL

The format of the JSONL index must be as follows:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair 'image_gcs_uri' is the Cloud Storage path to the image.
- The key/value pair 'classification_annotation' is the label field.
 - The key/value pair 'display_name' is the label

    { 'image_gcs_uri': image, 'classification_annotation': { 'display_name': label } }
    
*Note*: The dictionary key fields may alternatively be in camelCase. For example, 'image_gcs_uri' can also be 'imageGcsUri'.

#### Dataset splitting

The index files may include information about data splitting.

##### CSV

Each row entry in a CSV index file can be preceded by a first column that indicates whether the data is part of the training (TRAINING), test (TEST) or validation (VALIDATION) data. Alternatively, AI Platform (Unified) supports the CAIP (pre-AI Platform (Unified)) version of the tags: TRAIN, TEST and VALIDATE. For example:

    TRAINING, "this is the data item", "this is the label"
    TEST, "this is the data item", "this is the label"
    VALIDATION, "this is the data item", "this is the label"
    
##### JSONL

Each object entry in a JSONL index file can have a 'ml_use' key/value pair that indicates whether the data is part of the training (training), test (test) or validation (validation) data.

    { 'image_gcs_uri': image, 'classification_annotation': { 'display_name': label }, 'data_item_resource_labels':{'aiplatform.googleapis.com/ml_use':'training'} }
    
If the index does not contain data splitting information, AI Platform will automatically split the dataset for you.

### Import data

We have preprocessed the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) to the import format required by AI Platform.


In [19]:
IMPORT_FILE = 'gs://cloud-samples-data/vision/automl_classification/flowers/flowers.jsonl'

In [20]:
!gsutil cat -r 0-441 {IMPORT_FILE}

{'image_gcs_uri': 'gs://cloud-ml-data/img/flower_photos/daisy/10140303196_b88d3d6cec.jpg', 'classification_annotation': {'display_name': 'daisy'}}
{'image_gcs_uri': 'gs://cloud-ml-data/img/flower_photos/daisy/10172379554_b296050f82_n.jpg', 'classification_annotation': {'display_name': 'daisy'}}
{'image_gcs_uri': 'gs://cloud-ml-data/img/flower_photos/daisy/10172567486_2748826a8b.jpg', 'classification_annotation': {'display_name': 'daisy'}}

To import the data, call the `import_data` method exposed by the dataset client.

In [21]:
config = [{
    'gcs_source': {'uris': [IMPORT_FILE]},
    'import_schema_uri': IMPORT_SCHEMA_IMAGE_CLASSIFICATION
}]

operation = dataset_client.import_data(name=dataset_name, import_configs=config)
print("Long running operation:", operation.operation.name)
response = operation.result()
dataset_name = response.name
print(dataset_name)

Long running operation: projects/895222332033/locations/us-central1/datasets/7399009567490703360/operations/6158802232801230848



### Get dataset information

You can list all datasets in your project and retrieve detailed information about a specific dataset using the `list_datasets` and `get_datasets` methods.

In [22]:
response = dataset_client.list_datasets(parent=PARENT)
for dataset in response:
    print(dataset.display_name, ' ', dataset.name)

flowers-dataset   projects/895222332033/locations/us-central1/datasets/7399009567490703360
flowers   projects/895222332033/locations/us-central1/datasets/4298281224046116864


In [24]:
dataset_name = 'projects/895222332033/locations/us-central1/datasets/4298281224046116864'

In [25]:
response = dataset_client.get_dataset(name=dataset_name)
print(response)

name: "projects/895222332033/locations/us-central1/datasets/4298281224046116864"
display_name: "flowers"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
create_time {
  seconds: 1612281952
  nanos: 900537000
}
update_time {
  seconds: 1612281953
  nanos: 794516000
}
etag: "AMEw9yM4fclVFz3eckE_96K2i6ra3C7WxlHGB9Ub5qoKJpemdSiP-lZ0K5NodKjccxpx"
labels {
  key: "aiplatform.googleapis.com/dataset_metadata_schema"
  value: "IMAGE"
}
metadata {
  struct_value {
    fields {
      key: "dataItemSchemaUri"
      value {
        string_value: "gs://google-cloud-aiplatform/schema/dataset/dataitem/image_1.0.0.yaml"
      }
    }
    fields {
      key: "gcsBucket"
      value {
        string_value: "cloud-ai-platform-dd92d61a-4628-4e8c-a94b-43eb3cee0e5f"
      }
    }
  }
}



### List data items

To retrieve the dataset's data items you can use the `list_data_items` method. 

In [26]:
count = 0
response = dataset_client.list_data_items(parent=dataset_name)
    
for data_item in response:
    count += 1

print('Number of images: {}'.format(count))
print('An example of item specification:')
print(data_item)


Number of images: 3666
An example of item specification:
name: "projects/895222332033/locations/us-central1/datasets/4298281224046116864/dataItems/9213637801582835345"
create_time {
  seconds: 1612282847
  nanos: 515282000
}
payload {
  struct_value {
    fields {
      key: "gcsUri"
      value {
        string_value: "gs://cloud-ml-data/img/flower_photos/tulips/8722514702_7ecc68691c.jpg"
      }
    }
    fields {
      key: "mimeType"
      value {
        string_value: "image/jpeg"
      }
    }
  }
}
update_time {
  seconds: 1612282847
  nanos: 515282000
}
etag: "AMEw9yP5LkZJxkumaL8F1mEyUx6qoslwjrsCBURnIedfUhfy5G0YGpMb-DVNiEyfTXi0"



## Training a model 

The dataset is ready so we can move on to configuring a training job. There are three methods of training a custom model in AI Platform:
* [Custom jobs](https://cloud.google.com/ai-platform-unified/docs/training/create-custom-job)
* [Hyperparameter tuning jobs](https://cloud.google.com/ai-platform-unified/docs/training/using-hyperparameter-tuning)
* [Training pipelines](https://cloud.google.com/ai-platform-unified/docs/training/create-training-pipeline)

A training pipeline encapsulates  additional steps in addition to a training step, specifically: accessing data from an AI Platform dataset and uploading the trained model to AI Platform. The training step of a training pipeline can be either a Custom job or a Hyperparameter tuning job. 

In this sample, we utilize a training pipeline with a custom job training step.

There are two ways you can configure a custom training job:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

You will use the second method.

### Create a training container image
#### Create a folder for training images source code and configs

In [27]:
! rm -fr trainer; mkdir trainer
! touch trainer/__init__.py

#### Define the training script. 

In the next cell, you will write the contents of the training script to the `task.py` file.

Review the script. In summary, the script uses transfer learning to train an image classification model. The model uses the pre-trained ResNet50 as a base and a simple FCNN classifiction top. The script builds a `tf.data` data ingestion pipeline using the AI Platform dataset as a source. Notice how the AI Platform dataset is passed to the script.

At runtime, AI Platform passes metadata about your dataset to your training application by setting the following environment variables in your training container.

* AIP_DATA_FORMAT: The format that your dataset is exported in. Possible values include: jsonl, csv, or bigquery.
* AIP_TRAINING_DATA_URI: The location that your training data is stored at.
* AIP_VALIDATION_DATA_URI: The location that your validation data is stored at.
* AIP_TEST_DATA_URI: The location that your test data is stored at.

If the AIP_DATA_FORMAT of your dataset is jsonl or csv, the data URI values refer to Cloud Storage URIs, like `gs://bucket_name/path/training-*`. To keep the size of each data file relatively small, AI Platform splits your dataset into multiple files. Because your training, validation, or test data may be split into multiple files, the URIs are provided in wildcard format.

Image datasets are passed to your training application in JSONL format. The schema is the same as the schema of the index used to import the image data.

Refer to [AI Platform (Unified)](https://cloud.google.com/ai-platform-unified/docs/training/using-managed-datasets) for more information about using managed datasets in a custom training application.

In [28]:
%%writefile trainer/task.py
# Copyright 2020 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import argparse
import os
import time

#import hypertune
import numpy as np
import pandas as pd
import tensorflow as tf


IMG_HEIGHT = 224
IMG_WIDTH = 224
AUTOTUNE = tf.data.experimental.AUTOTUNE

    
def build_model(num_layers, dropout_ratio, num_classes):
    """
    Creates a custom image classificatin model using ResNet50 
    as a base model.
    """
    
    # Create the base model
    IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
    base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet',
                                                   pooling='avg')
    base_model.trainable = False
    
    # Add preprocessing and classification head
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.Dense(num_layers, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_ratio)(x)
    outputs = tf.keras.layers.Dense(num_classes)(x)
    
    # Assemble the model
    model = tf.keras.Model(inputs, outputs)
    
    # Compile the model
    base_learning_rate = 0.0001
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    return model


def image_dataset_from_aip_jsonl(pattern, class_names=None, img_height=224, img_width=224):
        """
        Generates a `tf.data.Dataset` from a set of JSONL files
        in the AI Platform image dataset index format. 
        
        Arguments:
            pattern: A wildcard pattern for a list of JSONL files.
                E.g. gs://bucket/folder/training-*.
            class_names: the list of class names that are expected
                in the passed index. 
            img_height: The height of a generated image
            img_width: The width of a generated image
            
        """
        
        def _get_label(class_name):
            """
            Converts a string class name to an integer label.
            """
            one_hot = class_name == class_names
            return tf.argmax(one_hot)
        
        def _decode_img(file_path):
            """
            Loads an image and converts it to a resized 3D tensor.
            """
            
            img = tf.io.read_file(file_path)
            img = tf.io.decode_image(img, 
                                     expand_animations=False)
            img = tf.image.resize(img, [img_height, img_width])
            
            return img
            
        def _process_example(file_path, class_name):
            """
            Creates a converted image and a class label from
            an image path and class name.
            """
            
            label = _get_label(class_name)
            img = _decode_img(file_path)
            
            return img, label
        
        # Read the JSONL index to a pandas DataFrame
        df = pd.concat(
            [pd.read_json(path, lines=True) for path in tf.io.gfile.glob(pattern)],
            ignore_index=True
        )
        
        # Parse classifcationAnnotations field
        df = pd.concat(
            [df, pd.json_normalize(df['classificationAnnotations'].apply(pd.Series)[0])], axis=1)
        
        paths = df['imageGcsUri'].values
        labels = df['displayName'].values
        inferred_class_names = np.unique(labels)
        
        if class_names is not None:
            class_names = np.array(class_names).astype(str)
            if set(inferred_class_names) != set(class_names):
                raise ValueError(
                    'The `class_names` passed does not match the '
                    'names in the image index '
                    'Expected: %s, received %s' %
                    (inferred_class_names, class_names))
            
        class_names = tf.constant(inferred_class_names)
        
        dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
        dataset = dataset.shuffle(len(labels), reshuffle_each_iteration=False)
        dataset = dataset.map(_process_example, num_parallel_calls=AUTOTUNE)
        dataset.class_names = class_names
        
        return dataset


def get_datasets(batch_size, img_height, img_width):
    """
    Creates training and validation splits as tf.data datasets
    from an AI Platform Dataset passed by the training pipeline.
    """
    
    def _configure_for_performance(ds):
        """
        Optimizes the performance of a dataset.
        """
        ds = ds.cache()
        ds = ds.prefetch(buffer_size=AUTOTUNE)
        return ds
    
    if  os.environ['AIP_DATA_FORMAT'] != 'jsonl':
        raise RuntimeError('Wrong dataset format: {}. Expecting - jsonl'.format(
            os.environ['AIP_DATA_FORMAT']))
        
    train_ds = image_dataset_from_aip_jsonl(
        pattern=os.environ['AIP_TRAINING_DATA_URI'],
        img_height=img_height,
        img_width=img_width)
    
    class_names = train_ds.class_names.numpy()
        
    valid_ds = image_dataset_from_aip_jsonl(
        pattern=os.environ['AIP_VALIDATION_DATA_URI'],
        class_names=class_names,
        img_height=img_height,
        img_width=img_width)
    
    train_ds = _configure_for_performance(train_ds.batch(batch_size))
    valid_ds = _configure_for_performance(valid_ds.batch(batch_size))
    
    return train_ds, valid_ds, class_names
        
    
def get_args():
    """
    Returns parsed command line arguments
    """
    
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--num-epochs',
        type=int,
        default=20,
        help='number of times to go through the data, default=20')
    parser.add_argument(
        '--batch-size',
        default=32,
        type=int,
        help='number of records to read during each training step, default=32')
    parser.add_argument(
        '--num-layers',
        default=64,
        type=int,
        help='number of hidden layers in the classification head , default=64')
    parser.add_argument(
        '--dropout-ratio',
        default=0.5,
        type=float,
        help='dropout ration in the classification head , default=128')
    parser.add_argument(
        '--model-dir',
        type=str,
        default='/tmp/saved_model',
        help='model dir , default=/tmp/saved_model')

    args, _ = parser.parse_known_args()
    return args


if __name__ == '__main__':
    
    if 'AIP_DATA_FORMAT' not in os.environ:
        raise RuntimeError('No dataset information available.')
   
    args = get_args()
    
    # Create the datasets and the model
    train_ds, valid_ds, class_names = get_datasets(args.batch_size, IMG_HEIGHT, IMG_WIDTH)
    model = build_model(args.num_layers, args.dropout_ratio, len(class_names))
    print(model.summary())
    
    # Start training
    history = model.fit(x=train_ds, 
                        validation_data=valid_ds, 
                        epochs=args.num_epochs)
    
    # Save the model
    if 'AIP_MODEL_DIR' in os.environ:
        model_dir = os.environ['AIP_MODEL_DIR']
    else:
        model_dir = args.model_dir
    print('Saving the model to: {}'.format(model_dir))
    model.save(model_dir)



Writing trainer/task.py


Create a Dockefile

In [29]:
dockerfile = f'''
FROM {TRAIN_BASE_IMAGE}

RUN pip install -U 'h5py<3.0.0'

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile)

Build the container image

In [30]:
train_image = f'gcr.io/{PROJECT_ID}/image_classifier'

In [31]:
! docker build -t {train_image} .

Sending build context to Docker daemon    172kB
Step 1/5 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-3:latest
 ---> 1da8084e22d8
Step 2/5 : RUN pip install -U 'h5py<3.0.0'
 ---> Using cache
 ---> 7931e988e970
Step 3/5 : WORKDIR /
 ---> Using cache
 ---> cd555d186ab4
Step 4/5 : COPY trainer /trainer
 ---> Using cache
 ---> f2945485392b
Step 5/5 : ENTRYPOINT ["python", "-m", "trainer.task"]
 ---> Using cache
 ---> 13edf65585ec
Successfully built 13edf65585ec
Successfully tagged gcr.io/jk-mlops-dev/image_classifier:latest


Push the image to Container Registry

In [32]:
! docker push {train_image}

The push refers to repository [gcr.io/jk-mlops-dev/image_classifier]

ce8ed86e: Preparing 
4b522827: Preparing 
ab07d59c: Preparing 
178561e2: Preparing 
28ecdaf6: Preparing 
323505c6: Preparing 
bf178d6b: Preparing 
c5334f43: Preparing 
8f1521c3: Preparing 
21c79fd1: Preparing 
2f63e1cf: Preparing 
7d3dd7ec: Preparing 
38633fe1: Preparing 
f29b440a: Preparing 
4b39f2b3: Preparing 
6711628b: Preparing 
f5975ab3: Preparing 
17ba195d: Preparing 
bec4097a: Preparing 
00c31be3: Preparing 
18b890fc: Preparing 
a7c9e3d1: Preparing 
4dce1444: Preparing 
30bcc944: Preparing 
e116c0c0: Preparing 
4df0ad6c: Preparing 
df553184: Preparing 
02706667: Layer already exists latest: digest: sha256:c9d07b760aa3081865351e6acf86e37f7c96f320286ff8b86185bd4167267b8c size: 6197


### Define the training pipeline

You will define a training pipeline that uses the Flowers dataset as input, runs training as a custom training job, and uploads the trained model to AI Platform. 

Let's start by assembling the custom training job specification.

#### Define container and worker pool specs

Your custom training job will use a custom training container created in the previous step. Recall that the training scripts can be configured through command line parameters. For example you can set a number of training epochs.

In this sample, we will run training on a single GPU equipped node.


In [33]:
epochs = 5

container_spec = {
    "image_uri": train_image, 
    "args": [
        "--num-epochs=" + str(epochs),
    ],
}

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_GPU,
            "accelerator_count": TRAIN_NGPU
        },
        "container_spec": container_spec,
    }
]

#### Assembly the job spec

Let's now assemble the custom job specification. 

After the custom job completes, the training pipeline finds the model artifacts that your training application creates in the output directory you specified for your Cloud Storage bucket. It uses these artifacts to create a model resource, which sets you up for model deployment.

There are two different ways to set the location for your model artifacts:

* If you set a `baseOutputDirectory` for your training job, make sure your training code saves your model artifacts to that location, using the `AIP_MODEL_DIR` environment variable set by AI Platform. After the training job is completed, AI Platform searches for the resulting model artifacts in `gs://BASE_OUTPUT_DIRECTORY/model`.
* If you set the `modelToUpload.artifactUri` field, the training pipeline uploads the model artifacts from that URI. You must set this field if you didn't set `baseOutputDirectory`.

If you specify both `baseOutputDirectory` and `modelToUpload.artifactUri`, AI Platform uses `modelToUpload.artifactUri`.

In [34]:
pipeline_display_name = 'image-classifier-pipeline'
base_output_dir = 'gs://{}/{}/'.format(BUCKET_NAME, pipeline_display_name)
job_spec =  {
    "worker_pool_specs": worker_pool_spec,
    "base_output_directory": {
        "output_uri_prefix": base_output_dir
    }
}
job_spec

{'worker_pool_specs': [{'replica_count': 1,
   'machine_spec': {'machine_type': 'n1-standard-4',
    'accelerator_type': <AcceleratorType.NVIDIA_TESLA_K80: 1>,
    'accelerator_count': 1},
   'container_spec': {'image_uri': 'gcr.io/jk-mlops-dev/image_classifier',
    'args': ['--num-epochs=5']}}],
 'base_output_directory': {'output_uri_prefix': 'gs://jk-ucaip-demos/image-classifier-pipeline/'}}

#### Assemble the pipeline specification

You can now assemble the pipeline specification.

In [35]:
model_display_name = pipeline_display_name + '-model'
dataset_id = dataset.name.split('/')[-1]

training_task_inputs = json_format.ParseDict(job_spec,Value())

training_pipeline_spec = {
    'display_name': pipeline_display_name,
    'input_data_config': {
        'dataset_id': dataset_id,
        'annotation_schema_uri': ANNOTATION_SCHEMA,
        'gcs_destination': {
            'output_uri_prefix': base_output_dir
        },
        'fraction_split': {
            'training_fraction': 0.5,
            'validation_fraction': 0.2
        },
    },
    'training_task_definition': CUSTOM_TRAINING_TASK_DEFINITION,
    'training_task_inputs': training_task_inputs,
    'model_to_upload': {
        'display_name': model_display_name,
        'container_spec': {
            'image_uri': DEPLOY_IMAGE
        }
    }
}

training_pipeline_spec

{'display_name': 'image-classifier-pipeline',
 'input_data_config': {'dataset_id': '4298281224046116864',
  'annotation_schema_uri': 'gs://google-cloud-aiplatform/schema/dataset/annotation/image_classification_1.0.0.yaml',
  'gcs_destination': {'output_uri_prefix': 'gs://jk-ucaip-demos/image-classifier-pipeline/'},
  'fraction_split': {'training_fraction': 0.5, 'validation_fraction': 0.2}},
 'training_task_definition': 'gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml',
 'training_task_inputs': struct_value {
   fields {
     key: "base_output_directory"
     value {
       struct_value {
         fields {
           key: "output_uri_prefix"
           value {
             string_value: "gs://jk-ucaip-demos/image-classifier-pipeline/"
           }
         }
       }
     }
   }
   fields {
     key: "worker_pool_specs"
     value {
       list_value {
         values {
           struct_value {
             fields {
               key: "container_spec"

### Start the training pipeline 


In [37]:
pipeline = pipeline_client.create_training_pipeline(
    parent=PARENT, training_pipeline=training_pipeline_spec)

Get pipeline info.

In [40]:
response = pipeline_client.get_training_pipeline(name=pipeline.name)
print(response)

pipeline
 name: projects/895222332033/locations/us-central1/trainingPipelines/1865246709731295232
 display_name: image-classifier-pipeline
 state: PipelineState.PIPELINE_STATE_RUNNING
 create_time: 2021-02-04 01:49:56.435937+00:00
 start_time: 2021-02-04 01:49:56.606220+00:00
 end_time: None
 update_time: 2021-02-04 01:49:59.162069+00:00
 labels: {}
name: "projects/895222332033/locations/us-central1/trainingPipelines/1865246709731295232"
display_name: "image-classifier-pipeline"
input_data_config {
  dataset_id: "4298281224046116864"
  fraction_split {
    training_fraction: 0.5
    validation_fraction: 0.2
  }
  gcs_destination {
    output_uri_prefix: "gs://jk-ucaip-demos/image-classifier-pipeline/"
  }
  annotation_schema_uri: "gs://google-cloud-aiplatform/schema/dataset/annotation/image_classification_1.0.0.yaml"
}
training_task_definition: "gs://google-cloud-aiplatform/schema/trainingjob/definition/custom_task_1.0.0.yaml"
training_task_inputs {
  struct_value {
    fields {
      

The pipeline will take a few minutes to complete. You can check the status of the pipeline by inspecting the `state` property returned by a call to the `get_training_pipeline` method.


In [45]:
response = pipeline_client.get_training_pipeline(name=pipeline.name)
model_name = None
if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
    print("Training pipeline has not completed yet")
else:
    print("Training pipeline completed")
    print("Model ready for deployment:")
    print(response.model_to_upload)
    model_name = response.model_to_upload.name

Training pipeline completed
Model ready for deployment:
name: "projects/895222332033/locations/us-central1/models/3972192463526821888"
display_name: "image-classifier-pipeline-model"
predict_schemata {
}
container_spec {
  image_uri: "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest"
}



Before proceeding wait till the training pipeline completes.

## Deploying a model

Deploying an uploaded model is a two step process. First you need to create or select an AI Platform Prediction endpoint. After the endpoint is ready, you deploy a model to the endpoint.

Refer to AI Platform (Unified) [documentation](https://cloud.google.com/ai-platform-unified/docs/general/deployment) for more information about deploying models.


### Get model information

You can list uploaded models and get detailed information about a specific model using  the `list_model` and `get_model` methods.

In [47]:
request = {
    'parent': PARENT,
    'filter': 'display_name="{}"'.format(model_display_name)
}

response = model_client.list_models(request)

for model in response:
    print(model.name)

projects/895222332033/locations/us-central1/models/3972192463526821888


In [49]:
model_client.get_model(name=model_name)

name: "projects/895222332033/locations/us-central1/models/3972192463526821888"
display_name: "image-classifier-pipeline-model"
predict_schemata {
}
metadata {
}
training_pipeline: "projects/895222332033/locations/us-central1/trainingPipelines/1865246709731295232"
container_spec {
  image_uri: "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest"
}
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gzip"
supported_input_storage_formats: "file-list"
supported_output_storage_formats: "jsonl"
create_time {
  seconds: 1612403396
  nanos: 435937000
}
update_time {
  seconds: 1612404472
  nanos: 281719000
}
etag: "AMEw9yPz9xzV_YYKcLxIAJeH0zXJM4LEjYOyJdweuMfVVuH3h8Cj3YjFu7I9m-lL96wB"
supported_export_formats {
  id: "custom-trained"
  exportable_contents: ARTIFACT
}
artifact_uri: "gs://jk-ucaip-demos/image-classifier-p

### Create an endpoint


Creating an endpoint returns a long running operation, since it may take a few moments to provision the endpoint for serving. You call `response.result()`, which is a synchronous call and will return when the endpoint is ready. The helper function will return the AI Platform (Unified) fully qualified identifier for the endpoint -- `response.name`.


In [50]:
endpoint_display_name = 'flower_classifier_endpoint'

endpoint = {
    'display_name': endpoint_display_name
}

In [51]:
response = endpoint_client.create_endpoint(parent=PARENT, endpoint=endpoint)
print('Long running operation: ',  response.operation.name)
result = response.result(timeout=300)
endpoint_name = result.name
print(endpoint_name)

Long running operation:  projects/895222332033/locations/us-central1/endpoints/3754418792482996224/operations/4249275990796140544
projects/895222332033/locations/us-central1/endpoints/3754418792482996224


### List endpoints

In [52]:
for endpoint in endpoint_client.list_endpoints(parent=PARENT):
    print(endpoint)

name: "projects/895222332033/locations/us-central1/endpoints/3754418792482996224"
display_name: "flower_classifier_endpoint"
etag: "AMEw9yOGx4L1oCcZgk4PRS1NTE_v0bP13uG6y9CSfkeNJNrd8fmfvk3d2yWQKnfVJ5-T"
create_time {
  seconds: 1612404654
  nanos: 38546000
}
update_time {
  seconds: 1612404668
  nanos: 942577000
}



### Deploy model to the endpoint


#### Define model deployment specification

In [59]:
display_name = 'flower_classifier'

machine_spec = {
    'machine_type': DEPLOY_MACHINE_TYPE,
    'accelerator_count': 0,
}

deployed_model_spec = {
    'model': model_name,
    'display_name': display_name,
    'dedicated_resources': {
        'min_replica_count': 1,
        'machine_spec': machine_spec
    },
}

traffic_split={'0': 100}

#### Deploy the model

In [60]:
response = endpoint_client.deploy_model(endpoint=endpoint_name, 
                                            deployed_model=deployed_model_spec, 
                                            traffic_split=traffic_split)

print("Long running operation:", response.operation.name)
result = response.result()
deployed_model_id = result.deployed_model.id

Long running operation: projects/895222332033/locations/us-central1/endpoints/3754418792482996224/operations/2852034206404444160


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- Endpoint
- Cloud Storage Bucket

In [64]:
print(dataset_name)
print(model_name)
print(endpoint_name)
print(deployed_model_id)

projects/895222332033/locations/us-central1/datasets/4298281224046116864
projects/895222332033/locations/us-central1/models/3972192463526821888
projects/895222332033/locations/us-central1/endpoints/3754418792482996224
4390270764872368128


In [66]:
delete_dataset = True
delete_model = True
undeploy_model = True
delete_endpoint = True
delete_bucket = True

# Undeploye the model
try:
    if undeploy_model:
        endpoint_client.undeploy_model(endpoint=endpoint_name,
                                       deployed_model_id=deployed_model_id,
                                       traffic_split={})
except Exception as e:
    print(e)
  
# Delete the endpoint using the AI Platform (Unified) fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        endpoint_client.delete_endpoint(name=endpoint_name)
except Exception as e:
    print(e)
        
# Delete the model using the AI Platform (Unified) fully qualified identifier for the model
try:
    if delete_model:
        model_client.delete_model(name=model_name)
except Exception as e:
    print(e)
    

# Delete the dataset using the AI Platform (Unified) fully qualified identifier for the dataset
try:
    if delete_dataset:
        dataset_client.delete_dataset(name=dataset_name)
except Exception as e:
    print(e)

if delete_bucket and 'BUCKET_NAME' in globals():
    ! gsutil rm -r gs://$BUCKET_NAME

404 Endpoint `projects/895222332033/locations/us-central1/endpoints/3754418792482996224` is not found.
404 The Endpoint "projects/895222332033/locations/us-central1/endpoints/3754418792482996224" does not exist.
404 The Dataset "projects/895222332033/locations/us-central1/datasets/4298281224046116864" does not exist.
BucketNotFoundException: 404 gs://jk-ucaip-demos bucket does not exist.


## Code snippets